In [1]:
from IPython.display import display
from IPython.core.debugger import set_trace
import sqlalchemy as sa
import logging
import logging.config
import sys
import os

from api.database import DBConn
from api.models import Models, Seeder, Faker
from api.queries import QueryBuilder
from api import Config

In [2]:
config = Config()
DBConn()
models = Models()
display(models.schemas)

{'er1': {'Item': models.model_item.Item,
  'User': models.model_user.User,
  'Attachment': models.model_attachment.Attachment,
  'Task': models.model_task.Task,
  'UserOnTask': models.model_user_on_task.UserOnTask,
  'List': models.model_list.List,
  'Subtask': models.model_subtask.Subtask}}

In [3]:
logger = logging.getLogger('sqlalchemy.engine')
logger.setLevel(logging.INFO)

In [4]:
with DBConn.get_session() as session:
    q = session.query(models.er1['Task']).first()
    obj = q.__marshmallow__().dump(q)
    display(obj)

INFO:sqlalchemy.engine.base.Engine:select version()
INFO:sqlalchemy.engine.base.Engine:{}
INFO:sqlalchemy.engine.base.Engine:select current_schema()
INFO:sqlalchemy.engine.base.Engine:{}
INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
INFO:sqlalchemy.engine.base.Engine:{}
INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
INFO:sqlalchemy.engine.base.Engine:{}
INFO:sqlalchemy.engine.base.Engine:show standard_conforming_strings
INFO:sqlalchemy.engine.base.Engine:{}
INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.base.Engine:SELECT er1.task.name AS er1_task_name, er1.task.finish_date AS er1_task_finish_date, er1.task.description AS er1_task_description, er1.task.list_name AS er1_task_list_name 
FROM er1.task 
 LIMIT %(param_1)s
INFO:sqlalchemy.engine.base.Engine:{'param_1': 1}
INFO:sqlalchemy.engine.base.Engine:SELECT er1.subtask.id AS er1_subtask_id, er1.subtask.name AS e

{'name': 'Patricia Watts',
 'subtask_on': [1, 5, 7, 9, 16, 24, 32, 37, 38, 45],
 'finish_date': '2010-09-29T04:32:49',
 'user_on': [{'user_name': 'John Baker', 'task_name': 'Patricia Watts'},
  {'user_name': 'Sharon Ross', 'task_name': 'Patricia Watts'},
  {'user_name': 'Taylor Sanchez', 'task_name': 'Patricia Watts'},
  {'user_name': 'David Perez', 'task_name': 'Patricia Watts'}],
 'list': 'Erin Hale',
 'attachment_on': [1, 10],
 'description': 'Memory while decide service. Thus family western fight party bring man. But source could many TV when book under.'}

INFO:sqlalchemy.engine.base.Engine:ROLLBACK


In [8]:
criterion = {
    "field_name": "Task.name",
    "operator": "==",
    "field_value": obj["name"]
}

with DBConn.get_session() as session:
    builder = QueryBuilder(session)
    r = builder.fetch_data(models.er1['Task'], filter_by=[criterion])
    display(r)

INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.base.Engine:SELECT er1.task.name, er1.task.finish_date, er1.task.description, er1.task.list_name 
FROM er1.task 
WHERE er1.task.name = %(name_1)s
INFO:sqlalchemy.engine.base.Engine:{'name_1': 'Patricia Watts'}


[{'name': 'Patricia Watts',
  'finish_date': datetime.datetime(2010, 9, 29, 4, 32, 49),
  'description': 'Memory while decide service. Thus family western fight party bring man. But source could many TV when book under.',
  'list_name': 'Erin Hale'}]

INFO:sqlalchemy.engine.base.Engine:ROLLBACK


In [6]:
with DBConn.get_session() as session:
    builder = QueryBuilder(session)
    r = builder.fetch_one(models.er1['Task'], filter_by=[criterion])
    display(r)

INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.base.Engine:SELECT er1.task.name AS er1_task_name, er1.task.finish_date AS er1_task_finish_date, er1.task.description AS er1_task_description, er1.task.list_name AS er1_task_list_name 
FROM er1.task 
WHERE er1.task.name = %(name_1)s 
 LIMIT %(param_1)s
INFO:sqlalchemy.engine.base.Engine:{'name_1': 'Patricia Watts', 'param_1': 1}
INFO:sqlalchemy.engine.base.Engine:SELECT er1.subtask.id AS er1_subtask_id, er1.subtask.name AS er1_subtask_name, er1.subtask.task_name AS er1_subtask_task_name 
FROM er1.subtask 
WHERE %(param_1)s = er1.subtask.task_name
INFO:sqlalchemy.engine.base.Engine:{'param_1': 'Patricia Watts'}
INFO:sqlalchemy.engine.base.Engine:SELECT er1.user_on_task.user_name AS er1_user_on_task_user_name, er1.user_on_task.task_name AS er1_user_on_task_task_name 
FROM er1.user_on_task 
WHERE %(param_1)s = er1.user_on_task.task_name
INFO:sqlalchemy.engine.base.Engine:{'param_1': 'Patricia Watts'}
INFO:sqlalchem

{'name': 'Patricia Watts',
 'subtask_on': [1, 5, 7, 9, 16, 24, 32, 37, 38, 45],
 'finish_date': '2010-09-29T04:32:49',
 'user_on': [{'user_name': 'John Baker', 'task_name': 'Patricia Watts'},
  {'user_name': 'Sharon Ross', 'task_name': 'Patricia Watts'},
  {'user_name': 'Taylor Sanchez', 'task_name': 'Patricia Watts'},
  {'user_name': 'David Perez', 'task_name': 'Patricia Watts'}],
 'list': 'Erin Hale',
 'attachment_on': [1, 10],
 'description': 'Memory while decide service. Thus family western fight party bring man. But source could many TV when book under.'}

INFO:sqlalchemy.engine.base.Engine:ROLLBACK


In [17]:
next(iter(models['er1']['Task'].__table__.columns)).name

'name'